# 나이브 베이어스
해당 메일이 스팸메일인지 아닌지 예측하는 문제이다.

data에는 스팸의 여부와 메일 내용이 저장되어있다.

In [28]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [29]:
import codecs

# utf-8 디코딩 문제로 인해 해당 코드로 파일을 읽어왔다.
with codecs.open('data/sms_spam.csv', "r",encoding='utf-8', errors='ignore') as fdata:
    sms_raw = pd.read_csv(fdata)

In [30]:
sms_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5559 entries, 0 to 5558
Data columns (total 2 columns):
type    5559 non-null object
text    5559 non-null object
dtypes: object(2)
memory usage: 86.9+ KB


In [31]:
sms_raw['type'].value_counts()

ham     4812
spam     747
Name: type, dtype: int64

In [32]:
sms_raw.head(10) # 임의 출력

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
5,ham,Aiya we discuss later lar... Pick u up at 4 is...
6,ham,Are you this much buzy
7,ham,Please ask mummy to call father
8,spam,Marvel Mobile Play the official Ultimate Spide...
9,ham,"fyi I'm at usf now, swing by the room whenever"


In [33]:
sms_corpus = sms_raw['text'] #dataframe의 'text'의 정보를 따로 저장

In [34]:
# text의 전처리
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=5) # 5개 이하로 나온 단어 무시
X = vectorizer.fit_transform(sms_corpus) # 문서-단어 행렬 작성 후 X에 저장
# 이는 해당 단어의 총 출연 횟수를 text에 등장한 단어 순서에 맞게 저장한 CounterVector 이다.
X.shape

(5559, 1800)

In [35]:
print(X[:3]) # countervector 임의 출력
type(X)

  (0, 773)	1
  (0, 326)	1
  (0, 819)	1
  (0, 1693)	1
  (0, 656)	1
  (0, 698)	1
  (0, 162)	1
  (0, 1791)	1
  (0, 731)	1
  (1, 1514)	1
  (1, 1032)	1
  (1, 198)	1
  (1, 643)	1
  (2, 1139)	1
  (2, 1557)	1
  (2, 695)	1
  (2, 272)	1
  (2, 1101)	1
  (2, 309)	1
  (2, 468)	1
  (2, 135)	1
  (2, 137)	1
  (2, 773)	1


scipy.sparse.csr.csr_matrix

In [36]:
X = X.toarray() # 학습을 위해서는 X가 array나 ndarray의 형태로 존재해야 함
y = sms_raw['type'] # 스팸 여부를 저장

In [37]:
from sklearn.model_selection import train_test_split
# 훈련데이터 75%, 테스트 데이터 25%를 지정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [38]:
from sklearn.naive_bayes import GaussianNB as nb
cl = nb() # 가우시안 나이브 베이어스 모델 생성
cl.fit(X_train, y_train) # 학습

from sklearn import metrics
y_pred = cl.predict(X_test) # 학습된 모델을 이용하여 예측
print(metrics.classification_report(y_test, y_pred)) # 예측 결과 출력

             precision    recall  f1-score   support

        ham       0.98      0.85      0.91      1187
       spam       0.51      0.90      0.65       203

avg / total       0.91      0.86      0.87      1390



In [39]:
# 라플라스 추정
# 라플라스 추정을 위해 MultinomialNB를 사용
from sklearn.naive_bayes import MultinomialNB as nb
cl = nb() # 멀티노미얼 나이브 베이어스 모델 생성
cl.fit(X_train, y_train) # 학습

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
from sklearn import metrics
y_pred = cl.predict(X_test) # 학습된 모델을 이용하여 예측
print(metrics.classification_report(y_test, y_pred)) # 예측 결과 출력

             precision    recall  f1-score   support

        ham       0.99      0.99      0.99      1187
       spam       0.96      0.95      0.95       203

avg / total       0.99      0.99      0.99      1390



### 실습 예제
상단의 예제에서 주어진 email data를 train과 test데이터의 비율은 8:2로 설정한 뒤, 가우시안 나이브 베이어스 학습 모델을 이용하여 예측 결과를 출력하시오.